## Run this first if you want NDVI/NDWI basemaps

In [9]:
import ee

ee.Authenticate(force=True)


Successfully saved authorization token.


## Vibes

In [1]:
%load_ext autoreload
%autoreload 2

from geovibes.ui import GeoVibes

vibes = GeoVibes(
    start_date="2024-01-01", end_date="2025-01-01", enable_ee=True, verbose=False
)

In [ ]:
## Example adding CDL aquaculture
import ee

ee.Initialize()

cdl = (
    ee.ImageCollection("USDA/NASS/CDL")
    .filterDate("2023-01-01", "2023-12-31")
    .first()
    .select("cropland")
)


vibes.add_ee_layer(cdl, {}, name="CDL", opacity=0.7)

## CDL Aquaculture + Water Mask with Morphological Closing

In [2]:
import ee

ee.Initialize()

# Load CDL 2023
cdl = (
    ee.ImageCollection("USDA/NASS/CDL")
    .filterDate("2023-01-01", "2023-12-31")
    .first()
    .select("cropland")
)

# Binary mask: Aquaculture (92) OR Open Water (111)
water_mask = cdl.eq(92).Or(cdl.eq(111))

# Morphological closing: dilation then erosion (3x3 kernel)
# This fills small gaps and smooths boundaries
closed_mask = (
    water_mask.focal_max(
        radius=1, kernelType="square", units="pixels"
    ).focal_min(radius=1, kernelType="square", units="pixels")  # dilation  # erosion
)

# Visualization: show the mask
mask_vis = {
    "min": 0,
    "max": 1,
    "palette": ["#00000000", "#0066FF"],  # transparent for 0, blue for 1
}

# Add both layers to compare
vibes.add_ee_layer(water_mask.selfMask(), mask_vis, name="CDL Water (raw)", opacity=0.7)
vibes.add_ee_layer(
    closed_mask.selfMask(), mask_vis, name="CDL Water (closed)", opacity=0.7
)

## Example adding ESRI LULC

In [2]:
import ee

ee.Initialize()
ee.Authenticate()
lulc = ee.ImageCollection(
    "projects/sat-io/open-datasets/landcover/ESRI_Global-LULC_10m_TS"
)

# Get 2024 mosaic and remap to sequential values
lulc_2024 = (
    lulc.filterDate("2024-01-01", "2024-12-31")
    .mosaic()
    .remap([1, 2, 4, 5, 7, 8, 9, 10, 11], [1, 2, 3, 4, 5, 6, 7, 8, 9])
)

# Visualization: Water, Trees, Flooded Veg, Crops, Built, Bare, Snow, Clouds, Rangeland
vis_params = {
    "min": 1,
    "max": 9,
    "palette": [
        "#1A5BAB",  # Water
        "#358221",  # Trees
        "#87D19E",  # Flooded Vegetation
        "#FFDB5C",  # Crops
        "#ED022A",  # Built Area
        "#EDE9E4",  # Bare Ground
        "#F2FAFF",  # Snow/Ice
        "#C8C8C8",  # Clouds
        "#C6AD8D",  # Rangeland
    ],
}

vibes.add_ee_layer(lulc_2024, vis_params, name="LULC 2024", opacity=0.7)

## Example adding SRTM Elevation

In [10]:
# Add elevation layer from SRTM dataset
dataset = ee.Image("USGS/SRTMGL1_003")
elevation = dataset.select("elevation")

# Add elevation layer to the map
elevation_vis_params = {
    "min": 0,
    "max": 300,
    "palette": [
        "#000080",
        "#0000FF",
        "#00FFFF",
        "#00FF00",
        "#FFFF00",
        "#FF8000",
        "#FF0000",
        "#800000",
    ],  # Blue to red gradient for elevation
}

vibes.add_ee_layer(elevation, elevation_vis_params, name="SRTM Elevation", opacity=0.7)

## Example adding Cloud-Masked NDWI (Sentinel-2 with Cloud Score+)

In [ ]:
import ee

ee.Initialize()

# Cloud Score+ threshold (0-1, higher = more strict cloud masking)
CLOUD_THRESHOLD = 0.65

# Date range for composite
start_date = "2024-01-01"
end_date = "2024-12-31"

# Load Sentinel-2 SR and Cloud Score+ collections
s2_sr = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
cs_plus = ee.ImageCollection("GOOGLE/CLOUD_SCORE_PLUS/V1/S2_HARMONIZED")

# Join Cloud Score+ to S2 SR by system:index
s2_with_cs = s2_sr.linkCollection(cs_plus, ["cs"])

# Filter by date and cloud score
filtered = s2_with_cs.filterDate(start_date, end_date).map(
    lambda img: img.updateMask(img.select("cs").gte(CLOUD_THRESHOLD))
)


# Compute NDWI: (Green - NIR) / (Green + NIR)
# Sentinel-2: B3 = Green, B8 = NIR
def add_ndwi(img):
    ndwi = img.normalizedDifference(["B3", "B8"]).rename("NDWI")
    return img.addBands(ndwi)


ndwi_collection = filtered.map(add_ndwi)

# Create median composite
ndwi_composite = ndwi_collection.select("NDWI").median()

# Visualization: blue for water (positive NDWI), brown for land (negative)
ndwi_vis = {
    "min": -0.5,
    "max": 0.5,
    "palette": ["#8B4513", "#D2B48C", "#FFFFFF", "#87CEEB", "#0000FF"],
}

vibes.add_ee_layer(ndwi_composite, ndwi_vis, name="NDWI (Cloud-masked)", opacity=0.7)